In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import re
pd.set_option('display.max_rows', 500)

In [2]:
# If we only want high security prisons
# high_security = ["CAC", "CCI", "COR", "LAC", "SAC", "HDSP", "KVSP", "PBSP", "SVSP", "SATF"]
# df_highsecurity = df.drop(df[~df['Institution'].isin(high_security)].index, axis=0)


In [3]:
# Create a new data type for months in order to sort by month
months_dtype = CategoricalDtype(categories=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'], ordered=True)

In [4]:
# Load in 2009-2019 data
df_2009_2019 = pd.read_csv("finished-csvs/finished-2009-2019-everything.csv", 
                           encoding="utf-8", dtype={'Month':months_dtype})
df_2009_2019.head()

,Institution,Year,Top Category,Subcategory,Metric Title,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,CCWF,2009,Custody Operations,Total Bed Capacity,Contract Beds,0,0,0,0,0,0,0,0,0,0,0,0
1,CCWF,2009,Custody Operations,Total Bed Capacity,Design Beds,1916,1953,1945,1961,1969,1999,2007,1986,1966,2006,1969,1993
2,CCWF,2009,Custody Operations,Total Bed Capacity,Non-Traditional Beds,265,62,72,72,36,0,0,72,67,0,0,0
3,CCWF,2009,Custody Operations,Total Bed Capacity,Overcrowding Beds,1801,1886,1851,1884,1894,1897,1927,1919,1884,1926,1902,1920
4,CCWF,2009,Custody Operations,Total Bed Capacity,Temporary Beds (FamilyVisiting/Tank Beds),0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Drop Top Category column, which doesn't exist in later datasets
df_2009_2019_no_top = df_2009_2019.drop(columns=['Top Category'], axis=1)
df_2009_2019_no_top.head()

,Institution,Year,Subcategory,Metric Title,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,CCWF,2009,Total Bed Capacity,Contract Beds,0,0,0,0,0,0,0,0,0,0,0,0
1,CCWF,2009,Total Bed Capacity,Design Beds,1916,1953,1945,1961,1969,1999,2007,1986,1966,2006,1969,1993
2,CCWF,2009,Total Bed Capacity,Non-Traditional Beds,265,62,72,72,36,0,0,72,67,0,0,0
3,CCWF,2009,Total Bed Capacity,Overcrowding Beds,1801,1886,1851,1884,1894,1897,1927,1919,1884,1926,1902,1920
4,CCWF,2009,Total Bed Capacity,Temporary Beds (FamilyVisiting/Tank Beds),0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201282,SVSP,2019,Adverse Actions,Per 100 Staff,0.44,0.09,0.51,0.00,0.00,0.26,0.69,0.09,0.35,0.34,0.60,0.51
201283,SVSP,2019,Adverse Actions,Dismissals (Non-Medical),0,1,1,0,0,0,0,0,0,0,0,0
201284,SVSP,2019,Adverse Actions,Adverse Actions Total (Medical),2,1,1,1,0,0,0,0,0,1,0,0
201285,SVSP,2019,Adverse Actions,Per 100 Staff,0.17,0.09,0.09,0.08,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.00


In [6]:
# Melt the 2009-2019 to a long format
df_2009_2019_melted = pd.melt(df_2009_2019_no_top, 
                              id_vars=['Institution','Subcategory','Metric Title','Year'], 
                              value_name='Value', var_name='Month', ignore_index=True)
df_2009_2019_melted.head()

,Institution,Subcategory,Metric Title,Year,Month,Value
0,CCWF,Total Bed Capacity,Contract Beds,2009,Jan,0
1,CCWF,Total Bed Capacity,Design Beds,2009,Jan,1916
2,CCWF,Total Bed Capacity,Non-Traditional Beds,2009,Jan,265
3,CCWF,Total Bed Capacity,Overcrowding Beds,2009,Jan,1801
4,CCWF,Total Bed Capacity,Temporary Beds (FamilyVisiting/Tank Beds),2009,Jan,0
...,...,...,...,...,...,...
2415439,SVSP,Adverse Actions,Per 100 Staff,2019,Dec,0.51
2415440,SVSP,Adverse Actions,Dismissals (Non-Medical),2019,Dec,0
2415441,SVSP,Adverse Actions,Adverse Actions Total (Medical),2019,Dec,0
2415442,SVSP,Adverse Actions,Per 100 Staff,2019,Dec,0.00


In [7]:
# Convert month to a categorical type
df_2009_2019_melted = df_2009_2019_melted.astype({'Month':months_dtype})
df_2009_2019_melted.dtypes

Institution       object
Subcategory       object
Metric Title      object
Year               int64
Month           category
Value             object
dtype: object

In [8]:
# Read 2020-2023 population data
df_2020_2023_POP = pd.read_csv("finished-csvs/finished-2020-2023-population.csv", encoding="utf-8", dtype={'Month':months_dtype})
df_2020_2023_POP.head()

,Institution,Category,Subcategory,Metric Title,Month,Year,Value
0,CCWF,Custody Operations,Total Bed Capacity,Blueprint Crowding Capacity,Jan,2020,2964
1,CCWF,Custody Operations,Total Bed Capacity,Blueprint Crowding Capacity,Feb,2020,2964
2,CCWF,Custody Operations,Total Bed Capacity,Blueprint Crowding Capacity,Mar,2020,2964
3,CCWF,Custody Operations,Total Bed Capacity,Blueprint Crowding Capacity,Apr,2020,2964
4,CCWF,Custody Operations,Total Bed Capacity,Blueprint Crowding Capacity,May,2020,2964
...,...,...,...,...,...,...,...
874555,WSP,Administration,Adverse Actions,Dismissals (Medical) 2,Feb,2023,0
874556,WSP,Administration,Adverse Actions,Dismissals (Medical) 2,Mar,2023,0
874557,WSP,Administration,Adverse Actions,Dismissals (Medical) 2,Apr,2023,0
874558,WSP,Administration,Adverse Actions,Dismissals (Medical) 2,May,2023,0


In [9]:
# drop category column, as it doesn't exist in other datasets for concatenation
df_2020_2023_POP_no_cat = df_2020_2023_POP.drop(columns=['Category'], axis=1)
df_2020_2023_POP_no_cat.head()

,Institution,Subcategory,Metric Title,Month,Year,Value
0,CCWF,Total Bed Capacity,Blueprint Crowding Capacity,Jan,2020,2964
1,CCWF,Total Bed Capacity,Blueprint Crowding Capacity,Feb,2020,2964
2,CCWF,Total Bed Capacity,Blueprint Crowding Capacity,Mar,2020,2964
3,CCWF,Total Bed Capacity,Blueprint Crowding Capacity,Apr,2020,2964
4,CCWF,Total Bed Capacity,Blueprint Crowding Capacity,May,2020,2964
...,...,...,...,...,...,...
874555,WSP,Adverse Actions,Dismissals (Medical) 2,Feb,2023,0
874556,WSP,Adverse Actions,Dismissals (Medical) 2,Mar,2023,0
874557,WSP,Adverse Actions,Dismissals (Medical) 2,Apr,2023,0
874558,WSP,Adverse Actions,Dismissals (Medical) 2,May,2023,0


In [10]:
# Read the incident data from same year as population, which we'll merge
df_2020_2023_Incidents = pd.read_csv("finished-csvs/finished-2020-2023-incidents.csv", encoding="utf-8", dtype={'Month':months_dtype})
df_2020_2023_Incidents.head()

,Subcategory,Institution,Metric Title,Month,Year,Value
0,Number of Incidents,ASP,Number of Incidents,Jan,2020,22
1,Number of Incidents,ASP,Number of Incidents,Feb,2020,22
2,Number of Incidents,ASP,Number of Incidents,Mar,2020,17
3,Number of Incidents,ASP,Number of Incidents,Apr,2020,19
4,Number of Incidents,ASP,Number of Incidents,May,2020,17


In [11]:
# Drop any duplicate rows (with all column values except 'Value' duplicated) in each of these three,
# keeping the 2nd occurence as it is the more recent one
df_2009_2019_melted.drop_duplicates(subset=['Institution', 'Subcategory', 'Metric Title', 
                                            'Year', 'Month'], 
                                    inplace=True, ignore_index=True, keep = 'last')
df_2020_2023_POP_no_cat.drop_duplicates(subset=['Institution', 'Subcategory', 'Metric Title', 
                                            'Year', 'Month'], 
                                        inplace=True, ignore_index=True, keep='last')
df_2020_2023_Incidents.drop_duplicates(subset=['Institution', 'Subcategory', 'Metric Title', 
                                            'Year', 'Month'], 
                                       inplace=True, ignore_index=True, keep='last')

In [12]:
# Note: errant values in pop dataframe -- deal with this below
df_2020_2023_POP_no_cat.loc[(df_2020_2023_POP_no_cat['Institution'] == 'CAC') &
                           (df_2020_2023_POP_no_cat['Metric Title'] == 'Documented Use of Force')]

Institution         Subcategory  \
0              CCWF  Total Bed Capacity   
1              CCWF  Total Bed Capacity   
2              CCWF  Total Bed Capacity   
3              CCWF  Total Bed Capacity   
4              CCWF  Total Bed Capacity   
...             ...                 ...   
3350025         WSP       Mental Health   
3350026         WSP       Mental Health   
3350027         WSP       Mental Health   
3350028         WSP       Mental Health   
3350029         WSP       Mental Health   

                                          Metric Title  Year Month Value  
0                                        Contract Beds  2009   Jan     0  
1                                          Design Beds  2009   Jan  1916  
2                                 Non-Traditional Beds  2009   Jan   265  
3                                    Overcrowding Beds  2009   Jan  1801  
4            Temporary Beds (FamilyVisiting/Tank Beds)  2009   Jan     0  
...                                                ...   ...   ...   ...  
3350025  UOF Incidents Involving Mental Health Inmates  2023   Jun    24  
3350026  UOF Incidents Involving Mental Health Inmates  2023   Jul    22  
3350027  UOF Incidents Involving Mental Health Inmates  2023   Aug    17  
3350028  UOF Incidents Involving Mental Health Inmates  2023   Sep    23  
3350029  UOF Incidents Involving Mental Health Inmates  2023   Oct     9  

[3350030 rows x 6 columns]

In [13]:
# Add column indicating which file row came from
df_2009_2019_melted['file'] = 'df_2009_2019_melted'
df_2020_2023_POP_no_cat['file'] = 'df_2020_2023_POP_no_cat'
df_2020_2023_Incidents['file'] = 'df_2020_2023_Incidents'

# Row bind all dataframes together
df = pd.concat([df_2009_2019_melted, df_2020_2023_POP_no_cat, df_2020_2023_Incidents], 
               ignore_index=True, verify_integrity=True)

# Drop duplicates based on all columns except file name
df.drop_duplicates(subset=['Institution', 'Subcategory', 'Metric Title', 
                                            'Year', 'Month', 'Value'], 
                                       inplace=True, ignore_index=True, keep='first')

df.head()

,Institution,Subcategory,Metric Title,Year,Month,Value,file
0,CCWF,Total Bed Capacity,Contract Beds,2009,Jan,0,df_2009_2019_melted
1,CCWF,Total Bed Capacity,Design Beds,2009,Jan,1916,df_2009_2019_melted
2,CCWF,Total Bed Capacity,Non-Traditional Beds,2009,Jan,265,df_2009_2019_melted
3,CCWF,Total Bed Capacity,Overcrowding Beds,2009,Jan,1801,df_2009_2019_melted
4,CCWF,Total Bed Capacity,Temporary Beds (FamilyVisiting/Tank Beds),2009,Jan,0,df_2009_2019_melted


In [14]:
df_sorted = df.sort_values(by=['Year','Month','Institution','Subcategory','Metric Title'], 
                           ignore_index=True)
df_sorted.head()

,Institution,Subcategory,Metric Title,Year,Month,Value,file
0,ASP,ASU EOP Hub,Actual Population,2009,Jan,0,df_2009_2019_melted
1,ASP,ASU Overflow,Actual Population,2009,Jan,0,df_2009_2019_melted
2,ASP,Academic Programs -\nAttendance Tracking,Average # Days in School (DIS),2009,Jan,98,df_2009_2019_melted
3,ASP,Academic Programs -\nAttendance Tracking,Average Daily Attendance,2009,Jan,663,df_2009_2019_melted
4,ASP,Academic Programs -\nAttendance Tracking,Avg. Length of Time in Assignment(LTA),2009,Jan,154,df_2009_2019_melted


In [15]:
# See how many rows are duplicates based on all columns except 'Value'
dup = df_sorted[df_sorted.duplicated(subset=['Institution', 'Subcategory', 'Metric Title',
                                                  'Year', 'Month'],
                                    keep = False)]

In [16]:
# Most of the dups are with 2020_2023_pop file having value of '0' or '-'
# Check values where this is not true
dup.loc[(dup['file'] == 'df_2020_2023_POP_no_cat') & 
        (dup['Value'] != '0') & (dup['Value'] != '-')].head()

,Institution,Subcategory,Metric Title,Year,Month,Value,file
2257733,ASP,FMLA,Medical,2019,Dec,624,df_2020_2023_POP_no_cat
2257736,ASP,FMLA,Non-Custody,2019,Dec,939,df_2020_2023_POP_no_cat
2258096,ASP,Inmate Disciplinaries,Administrative RVRs,2019,Dec,11,df_2020_2023_POP_no_cat
2258105,ASP,Inmate Disciplinaries,Battery on a Prisoner,2019,Dec,7,df_2020_2023_POP_no_cat
2258109,ASP,Inmate Disciplinaries,Conduct,2019,Dec,30,df_2020_2023_POP_no_cat


In [17]:
dup.loc[(dup['file'] == 'df_2020_2023_POP_no_cat') & (dup['Value'] == '-')] # lots of '-' Value in 2020

,Institution,Subcategory,Metric Title,Year,Month,Value,file
2289769,ASP,Type of Force,Baton,2020,Jan,-,df_2020_2023_POP_no_cat
2289775,ASP,Type of Force,OC,2020,Jan,-,df_2020_2023_POP_no_cat
2289777,ASP,Type of Force,Other,2020,Jan,-,df_2020_2023_POP_no_cat
2289779,ASP,Type of Force,Physical Force,2020,Jan,-,df_2020_2023_POP_no_cat
2290435,CAC,Type of Force,Baton,2020,Jan,-,df_2020_2023_POP_no_cat
...,...,...,...,...,...,...,...
3350025,WSP,Type of Force,37 mm/40 mm,2020,Dec,-,df_2020_2023_POP_no_cat
3350026,WSP,Type of Force,Baton,2020,Dec,-,df_2020_2023_POP_no_cat
3350027,WSP,Type of Force,OC,2020,Dec,-,df_2020_2023_POP_no_cat
3350028,WSP,Type of Force,Other,2020,Dec,-,df_2020_2023_POP_no_cat


In [23]:
# Write final merged dataframe to CSV
df_final.to_csv("finished-csvs/all-data-combined-long-format.csv", encoding="utf-8", index=False)